# file_system

In [ ]:
open sm'_operators
open rust_operators

()



In [ ]:
//// test

open testing

()



## types

In [ ]:
inl types () =
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::fs::File\")>]\n#endif\ntype std_fs_File = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::fs::FileType\")>]\n#endif\ntype std_fs_FileType = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::path::Display\")>]\n#endif\ntype std_path_Display = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::path::Path\")>]\n#endif\ntype std_path_Path = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"std::path::PathBuf\")>]\n#endif\ntype std_path_PathBuf = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"async_walkdir::DirEntry\")>]\n#endif\ntype async_walkdir_DirEntry = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"async_walkdir::Filtering\")>]\n#endif\ntype async_walkdir_Filtering = class end"
    global "#if FABLE_COMPILER\n[<Fable.Core.Erase; Fable.Core.Emit(\"async_walkdir::WalkDir\")>]\n#endif\ntype async_walkdir_WalkDir = class end"

()



In [ ]:
inl types () =
    types ()
    date_time.types ()
    env.types ()
    rust.types ()
    sm'.types ()
    stream.types ()

()



## file_mode

In [ ]:
nominal file_mode' = $'System.IO.FileMode'

union file_mode =
    | ModeCreateNew
    | ModeCreate
    | ModeOpen
    | ModeOpenOrCreate
    | Truncate
    | Append

inl file_mode = function
    | ModeCreateNew => $'System.IO.FileMode.CreateNew' : file_mode'
    | ModeCreate => $'System.IO.FileMode.Create' : file_mode'
    | ModeOpen => $'System.IO.FileMode.Open' : file_mode'
    | ModeOpenOrCreate => $'System.IO.FileMode.OpenOrCreate' : file_mode'
    | Truncate => $'System.IO.FileMode.Truncate' : file_mode'
    | Append => $'System.IO.FileMode.Append' : file_mode'

()



## file_access

In [ ]:
nominal file_access' = $'System.IO.FileAccess'

union file_access =
    | AccessRead
    | AccessWrite
    | AccessReadWrite

inl file_access = function
    | AccessRead => $'System.IO.FileAccess.Read' : file_access'
    | AccessWrite => $'System.IO.FileAccess.ReadWrite' : file_access'
    | AccessReadWrite => $'System.IO.FileAccess.ReadWrite' : file_access'

()



## file_share

In [ ]:
nominal file_share' = $'System.IO.FileShare'

union file_share =
    | ShareNone
    | ShareRead
    | ShareWrite
    | ShareReadWrite
    | ShareDelete

inl file_share = function
    | ShareNone => $'System.IO.FileShare.None' : file_share'
    | ShareRead => $'System.IO.FileShare.Read' : file_share'
    | ShareWrite => $'System.IO.FileShare.Write' : file_share'
    | ShareReadWrite => $'System.IO.FileShare.ReadWrite' : file_share'
    | ShareDelete => $'System.IO.FileShare.Delete' : file_share'

()



## file_stream

In [ ]:
nominal file_stream' = $'System.IO.FileStream'

inl file_stream (path : string) mode access share : file_stream' =
    run_target function
        | Fsharp (Native) => fun () =>
            inl mode = mode |> file_mode
            inl access = access |> file_access
            inl share = share |> file_share
            $'new System.IO.FileStream (!path, !mode, !access, !share)'
        | _ => fun () => null ()

()



## display

In [ ]:
nominal display = $'std_path_Display'

()



## path

In [ ]:
nominal path = $'std_path_Path'
nominal path = $'std_path_Path'
nominal path_buf = $'std_path_PathBuf'

()



## new_path_buf

In [ ]:
inl new_path_buf (path : sm'.std_string) : path_buf =
    open rust_operators
    !\\(path, $'"std::path::PathBuf::from($0)"')

()



## path_buf_from

In [ ]:
inl path_buf_from (path : rust.box path) : path_buf =
    open rust_operators
    !\\(path, $'"std::path::PathBuf::from($0)"')

()



## path_buf_join

In [ ]:
inl path_buf_join (s : string) (path_buf : path_buf) : path_buf =
    open rust_operators
    !\\((path_buf, s |> sm'.to_std_string), $'"$0.join($1)"')

()



## path_buf_strip_prefix

In [ ]:
inl path_buf_strip_prefix (s : string) (path_buf : path_buf) : path_buf =
    open rust_operators
    !\\((path_buf, s |> sm'.to_std_string), $'"$0.strip_prefix($1).unwrap().to_path_buf()"')

()



## path_display

In [ ]:
inl path_display (path : rust.ref' path) : display =
    open rust_operators
    !\\(path, $'"$0.display()"')

()



## path_buf_display

In [ ]:
inl path_buf_display (path_buf : path_buf) : display =
    open rust_operators
    !\\(path_buf, $'"$0.display()"')

()



## path_buf_file_name

In [ ]:
inl path_buf_file_name (path : path_buf) : optionm'.option' (rust.ref' sm'.os_str) =
    open rust_operators
    !\($'"!path.file_name()"')

()



## path_buf_exists

In [ ]:
inl path_buf_exists (path_buf : path_buf) : bool =
    open rust_operators
    !\\(path_buf, $'"$0.exists()"')

()



## path_buf_is_dir

In [ ]:
inl path_buf_is_dir (path_buf : path_buf) : bool =
    open rust_operators
    !\\(path_buf, $'"$0.is_dir()"')

()



## path_buf_is_file

In [ ]:
inl path_buf_is_file (path_buf : path_buf) : bool =
    open rust_operators
    !\\(path_buf, $'"$0.is_file()"')

()



## path_buf_parent

In [ ]:
inl path_buf_parent (path_buf : path_buf) : optionm'.option' path_buf =
    open rust_operators
    !\\(path_buf, $'"$0.parent().map(std::path::PathBuf::from)"')

()



## ts_path_join

In [ ]:
inl ts_path_join (b : string) (a : string) : string =
    open ts_operators
    global "type IPathJoin = abstract join: [<System.ParamArray>] paths: string[] -> string"
    inl path : $'IPathJoin' = ts.import_all "path"

    inl a = join a
    inl b = join b
    !\($'"!path.join(!a, !b)"')

()



## (< />)

In [ ]:
let (</>) (a : string) (b : string) : string =
    run_target function
        | Fsharp (Native) => fun () =>
            $'System.IO.Path.Combine (!a, !b)'
        | Rust (Native) => fun () =>
            a
            |> sm'.to_std_string
            |> new_path_buf
            |> path_buf_join b
            |> path_buf_display
            |> sm'.format'
            |> sm'.from_std_string
        | TypeScript _ => fun () =>
            a |> ts_path_join b
        | _ => fun () => null ()

()



## directory_info

In [ ]:
nominal directory_info = $'System.IO.DirectoryInfo'

inl directory_info (path : string) : directory_info =
    path |> $'`directory_info '

()



## directory_info_exists

In [ ]:
inl directory_info_exists (info : directory_info) : bool =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!info.Exists'
        | _ => fun () => null ()

()



## directory_info_creation_time

In [ ]:
inl directory_info_creation_time (info : directory_info) : date_time.date_time =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!info.CreationTime'
        | _ => fun () => null ()

()



## directory_info_name

In [ ]:
inl directory_info_name (info : directory_info) : string =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!info.Name'
        | _ => fun () => null ()

()



## directory_info_full_name

In [ ]:
inl directory_info_full_name (info : directory_info) : string =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!info.FullName'
        | _ => fun () => null ()

()



## dir_entry

In [ ]:
nominal dir_entry = $'async_walkdir_DirEntry'

()



## walk_dir

In [ ]:
nominal walk_dir = $'async_walkdir_WalkDir'

()



## async_walkdir_filtering

In [ ]:
nominal async_walkdir_filtering = $'async_walkdir_Filtering'

()



## filtering

In [ ]:
union filtering =
    | Ignore
    | IgnoreDir
    | Continue

()



## stream_filter_map

In [ ]:
inl stream_filter_map forall t.
    (fn : resultm.result' dir_entry stream.io_error -> optionm'.option' t)
    (stream : walk_dir)
    : am'.vec t =

    inl fn = join fn
    inl result : am'.vec t =
        !\($'"futures_lite::stream::StreamExt::collect(futures_lite::stream::StreamExt::filter_map(!stream, |x| !fn(x))).await"')
    result

()



## new_walk_dir

In [ ]:
inl new_walk_dir (dir : string) : walk_dir =
    !\\(dir, $'"async_walkdir::WalkDir::new(&*$0)"')
    // inl walk_dir : walk_dir = walk_dir |> rust.to_mut
    // (!\($'"true; let mut !walk_dir = !walk_dir"') : bool) |> ignore

()



## walk_dir_filter

In [ ]:
inl walk_dir_filter (fn : dir_entry -> async.future_pin_send filtering) (walk_dir : walk_dir) : walk_dir =
    inl fn entry = async.future_init_send (2, 1) 0 fun () =>
        inl result = fn entry |> async.await_send
        inl filtering : async_walkdir_filtering =
            match result with
            | Ignore => !\($'"async_walkdir::Filtering::Ignore"')
            | IgnoreDir => !\($'"async_walkdir::Filtering::IgnoreDir"')
            | Continue => !\($'"async_walkdir::Filtering::Continue"')
        filtering
    !\\((walk_dir, fn), $'"async_walkdir::WalkDir::filter($0, |x| $1(x))"')

()



## file_type

In [ ]:
nominal file_type = $'std_fs_FileType'

()



## dir_entry_file_type

In [ ]:
inl dir_entry_file_type (dir_entry : dir_entry) : async.future_pin_send (resultm.result' file_type stream.io_error) =
    inl dir_entry = join dir_entry
    !\($'"Box::pin(async_walkdir::DirEntry::file_type(&!dir_entry))"')

()



## file_type_is_dir

In [ ]:
inl file_type_is_dir (file_type : file_type) : bool =
    inl file_type = join file_type
    !\($'"std::fs::FileType::is_dir(&!file_type)"')

()



## file

In [ ]:
nominal file = $'std_fs_File'

()



## file_open

In [ ]:
inl file_open (path : string) : resultm.result' file stream.io_error =
    !\($'"std::fs::File::open(&*!path)"')

()



## dir_entry_path

In [ ]:
inl dir_entry_path (dir_entry : dir_entry) : path_buf =
    !\\(dir_entry, $'"async_walkdir::DirEntry::path(&$0)"')

()



## get_temp_path

In [ ]:
inl get_temp_path () : string =
    run_target function
        | Fsharp (Native) => fun () =>
            $'System.IO.Path.GetTempPath' ()
        | Rust (Native) => fun () =>
            open rust_operators
            !\($'"std::env::temp_dir()"')
            |> path_buf_display
            |> sm'.format'
            |> sm'.from_std_string
        | _ => fun () => null ()

()



## get_file_name

In [ ]:
inl get_file_name (path : string) : string =
    run_target function
        | Fsharp (Native) => fun () =>
            path |> $'System.IO.Path.GetFileName'
        | Rust (Native) => fun () =>
            open rust_operators
            inl path_buf = path |> sm'.to_std_string |> new_path_buf
            !\\(path_buf, $'"$0.file_name()"')
            |> optionm'.unwrap
            |> sm'.from_os_str_ref
        | _ => fun () => null ()

()



## get_current_directory

In [ ]:
inl get_current_directory () : string =
    run_target function
        | Fsharp (Native) => fun () =>
            $'System.IO.Directory.GetCurrentDirectory' ()
        | Rust (Native) => fun () =>
            open rust_operators
            inl current_dir = !\($'"std::env::current_dir()"') : resultm.result' path_buf stream.io_error
            current_dir
            |> resultm.unwrap'
            |> path_buf_display
            |> sm'.format'
            |> sm'.from_std_string
        | _ => fun () => null ()

()



In [ ]:
//// test

get_current_directory ()

C:\home\git\polyglot\lib\spiral

let rec method0 () : string =
    let v0 : bool = true
    let mutable _v0 : string option = None 
    
#if FABLE_COMPILER || WASM || CONTRACT
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v1 : string = "std::env::current_dir()"
    let v2 : Result<std_path_PathBuf, std_io_Error> = Fable.Core.RustInterop.emitRustExpr () v1
    let v3 : string = "$0.unwrap()"
    let v4 : std_path_PathBuf = Fable.Core.RustInterop.emitRustExpr v2 v3
    let v5 : string = "$0.display()"
    let v6 : std_path_Display = Fable.Core.RustInterop.emitRustExpr v4 v5
    let v7 : bool = true
    let mutable _v7 : std_string_String option = None 
    
#if FABLE_COMPILER || WASM || CONTRACT
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v8 : string = "format!(\"{}\", $0)"
    let v9 : std_string_String = Fable.Core.RustInterop.emitRustExpr v6 v8
    v9 
    #endif
#if FABLE_COMPILER_RUST && WASM
    let v10 : string = "format!(\"{}\", $0)"
    let v11 : std_string_String = Fable.Core.RustIn

## directory_separator_char

In [ ]:
inl directory_separator_char () : char =
    run_target function
        | Fsharp (Native) => fun () =>
            $'System.IO.Path.DirectorySeparatorChar'
        | Rust (Native) => fun () =>
            open rust_operators
            !\($'"std::path::MAIN_SEPARATOR"')
        | _ => fun () => null ()

()



## normalize_path

In [ ]:
inl normalize_path (path : string) : string =
    inl path = path |> sm'.replace_regex @"^\\\\\?\\" ""
    $'$"{!path.[0] |> string |> _.ToLower()}{!path.[1..]}"' |> sm'.replace "\\" "/"

()



## get_full_path

In [ ]:
inl get_full_path (path : string) : string =
    run_target function
        | Fsharp (Native) => fun () =>
            inl path = join path
            path |> $'System.IO.Path.GetFullPath'
        | Rust (Native) => fun () =>
            inl path = join path
            open rust_operators
            inl path_buf = path |> sm'.to_std_string |> new_path_buf
            if path_buf |> path_buf_exists |> not then
                inl current_dir = get_current_directory ()
                current_dir </> path
                |> normalize_path
                |> sm'.split "/"
                |> fun x =>
                    ((a x : _ i32 _), (0i32, (a ;[] : _ i32 _)))
                    ||> am.foldBack fun x level, acc =>
                        match x, level with
                        | "..", _ => level + 1, acc
                        | ".", _ => level, acc
                        | _, 0 when x |> sm'.ends_with ":" => 0, a ;[ $'$"{!current_dir.[0]}:"' ] ++ acc
                        | _, 0 => 0, a ;[ x ] ++ acc
                        | _ => level - 1, acc
                |> snd
                |> seq.of_array'
                |> sm'.concat (directory_separator_char () |> sm'.obj_to_string)
            else
                inl path = !\\(path, $'"std::fs::canonicalize(&*$0)"') : resultm.result' path_buf stream.io_error
                path
                |> resultm.unwrap'
                |> path_buf_display
                |> sm'.format'
                |> sm'.from_std_string
        | _ => fun () => null ()

()



In [ ]:
//// test
//// print_code=false

"."
|> get_full_path
|> directory_info
|> directory_info_name
|> _assert_eq "spiral"

assert_eq / actual: "spiral" / expected: "spiral"


In [ ]:
//// test
//// print_code=false

"dir/.././._file"
|> get_full_path
|> _assert_eq (get_current_directory () </> "._file")

assert_eq / actual: "C:\home\git\polyglot\lib\spiral\._file" / expected: "C:\home\git\polyglot\lib\spiral\._file"


In [ ]:
//// test
//// print_code=false
///! rust -d regex

types ()
"."
|> get_full_path
|> sm'.to_std_string
|> new_path_buf
|> path_buf_file_name
|> optionm'.unwrap
|> sm'.from_os_str_ref
|> _assert_eq "spiral"

assert_eq / actual: "spiral" / expected: "spiral"


In [ ]:
//// test
//// print_code=false
///! rust -d regex

types ()
"dir/.././._file"
|> get_full_path
|> _assert_eq (get_current_directory () </> "._file")

assert_eq / actual: "C:\home\git\polyglot\lib\spiral\._file" / expected: "C:\home\git\polyglot\lib\spiral\._file"


## create_temp_directory_name

In [ ]:
inl create_temp_directory_name () =
    run_target function
        | Fsharp (Native)
        | Rust (Native) => fun () =>
            inl assembly_name = env.get_entry_assembly_name ()
            get_temp_path ()
            </> ($'$"\!{!assembly_name}"' : string)
            </> (date_time.now () |> date_time.new_guid_from_date_time |> sm'.obj_to_string)
        | _ => fun () => null ()

()



In [ ]:
//// test
//// print_code=false

types ()
create_temp_directory_name ()
|> _assert_contains (directory_separator_char ())

assert_contains / actual: "C:\Users\i574n\AppData\Local\Temp\!dotnet-repl\20240509-0459-4264-6491-600400c554df" / expected: '\\'


In [ ]:
//// test
///! rust -d chrono

types ()
create_temp_directory_name ()
|> _assert_contains (directory_separator_char ())

.rs output:
assert_contains / actual: "C:\Users\i574n\AppData\Local\Temp\!spiral_builder_3a464612f1751f26bb1fc91b720052fd2533cd805f9fe2606dbaeda909fbd76a\20240509-0500-1796-7695-000000718f43" / expected: '\\'




.fsx:
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::fs::File")>]
#endif
type std_fs_File = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::fs::FileType")>]
#endif
type std_fs_FileType = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::Display")>]
#endif
type std_path_Display = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::Path")>]
#endif
type std_path_Path = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::PathBuf")>]
#endif
type std_path_PathBuf = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::DirEntry")>]
#endif
type async_walkdir_DirEntry = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::Filtering")>]
#endif
type async_walkdir_Filtering = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::WalkDir")>]
#endif
type async_walkdir_WalkDir = class end
#if FABLE_COMPILER


## get_source_directory

In [ ]:
inl get_source_directory () =
    $'__SOURCE_DIRECTORY__' : string

()



In [ ]:
//// test

get_source_directory ()
|> directory_info
|> directory_info_name
|> _assert_eq "spiral"

let rec method1 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : string = __SOURCE_DIRECTORY__
    let v1 : (string -> System.IO.DirectoryInfo) = System.IO.DirectoryInfo 
    let v2 : System.IO.DirectoryInfo = v1 v0
    let v3 : bool = true
    let mutable _v3 : string option = None 
    
#if FABLE_COMPILER || WASM || CONTRACT
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v4 : string = null |> unbox<string>
    v4 
    #endif
#if FABLE_COMPILER_RUST && WASM
    let v5 : string = null |> unbox<string>
    v5 
    #endif
#if FABLE_COMPILER_RUST && CONTRACT
    let v6 : string = null |> unbox<string>
    v6 
    #endif
#if FABLE_COMPILER_TYPESCRIPT
    let v7 : string = null |> unbox<string>
    v7 
    #endif
#if FABLE_COMPILER_PYTHON
    let v8 : string = null |> unbox<string>
    v8 
    #endif
#else
    let v9 : string = v2.Name
    v9 
    #endif
    |> fun x -> _v3 <- Some x
    let v10 : string = _v3.Value
    let v12 : bool = v10 = "spiral"
    let v14

## create_dir_all

In [ ]:
inl create_dir_all (path : string) : resultm.result' () stream.io_error =
    !\\(path, $'"std::fs::create_dir_all(&*$0)"')

()



## create_directory

In [ ]:
inl create_directory (path : string) : directory_info =
    run_target function
        | Fsharp (Native) => fun () =>
            path |> $'System.IO.Directory.CreateDirectory'
        | _ => fun () => null ()

()



## directory_get_files

In [ ]:
inl directory_get_files (path : string) : array_base string =
    run_target function
        | Fsharp (Native) => fun () =>
            path |> $'System.IO.Directory.GetFiles'
        | _ => fun () => null ()

()



## directory_exists

In [ ]:
inl directory_exists (path : string) : bool =
    run_target function
        | Fsharp (Native) => fun () =>
            path |> $'System.IO.Directory.Exists'
        | Rust (Native) => fun () =>
            inl path = path |> sm'.to_std_string |> new_path_buf
            path_buf_exists path && path_buf_is_dir path
        | TypeScript (Native) => fun () =>
            global "type IFsExistsSync = abstract existsSync: path: string -> bool"
            open ts_operators
            inl fs : $'IFsExistsSync' = ts.import_all "fs"
            !\\((fs, path), $'"$0.existsSync($1)"')
        | _ => fun () => null ()

()



## directory_get_parent

In [ ]:
inl directory_get_parent (path : string) : optionm'.option' string =
    run_target function
        | Fsharp (Native) => fun () =>
            inl parent : directory_info = path |> $'System.IO.Directory.GetParent'
            if parent =. null ()
            then None
            else parent |> directory_info_full_name |> Some
        | Rust (Native) => fun () =>
            path
            |> sm'.to_std_string
            |> new_path_buf
            |> path_buf_parent
            |> optionm'.try'
            |> path_buf_display
            |> sm'.format'
            |> sm'.from_std_string
            |> Some
        | TypeScript _ => fun () =>
            global "type IPathDirname = abstract dirname: path: string -> string"
            inl fs : $'IPathDirname' = ts.import_all "path"
            ts.emit_expr path $'"!fs.dirname($0)"' |> Some
        | _ => fun () => null ()
    |> optionm'.box

()



## file_delete

In [ ]:
inl file_delete (path : string) : () =
    run_target function
        | Fsharp (Native) => fun () =>
            path |> $'System.IO.File.Delete'
        | _ => fun () => null ()

()



## file_move

In [ ]:
inl file_move (new_path : string) (old_path : string) : () =
    run_target function
        | Fsharp (Native) => fun () =>
            $'System.IO.File.Move (!old_path, !new_path)'
        | _ => fun () => null ()

()



## file_copy

In [ ]:
inl file_copy (new_path : string) (old_path : string) : () =
    run_target function
        | Fsharp (Native) => fun () =>
            $'System.IO.File.Copy (!old_path, !new_path, true)'
        | Rust (Native) => fun () =>
            open rust_operators
            inl new_path = join new_path
            !\\(old_path, $'"std::fs::copy(&*$0, &*!new_path)"')
            |> fun x => x : _ u64 stream.io_error
            |> resultm.unwrap'
            |> ignore
        | _ => fun () => null ()

()



## file_exists

In [ ]:
inl file_exists (path : string) : bool =
    run_target function
        | Fsharp (Native) => fun () =>
            path |> $'System.IO.File.Exists'
        | Rust (Native) => fun () =>
            inl path_buf = path |> sm'.to_std_string |> new_path_buf
            path_buf_exists path_buf && path_buf_is_file path_buf
        | TypeScript (Native) => fun () =>
            open ts_operators
            global "type IFsExistsSync = abstract existsSync: path: string -> bool"
            inl fs : $'IFsExistsSync' = ts.import_all "fs"
            !\\((fs, path), $'"$0.existsSync($1)"')
        | _ => fun () => null ()

()



## directory_delete

In [ ]:
inl directory_delete recursive (path : string) : () =
    run_target function
        | Fsharp (Native) => fun () =>
            $'System.IO.Directory.Delete (!path, !recursive)'
        | Rust (Native) => fun () =>
            inl path = join path
            if path |> directory_exists then
                open rust_operators
                if recursive
                then !\\(path, $'"std::fs::remove_dir_all(&*$0).unwrap()"')
                else !\\(path, $'"std::fs::remove_dir(&*$0).unwrap()"')
        | _ => fun () => null ()

()



## read_all_text_async

In [ ]:
inl read_all_text_async (path : string) : _ string =
    run_target function
        | Fsharp (Native) => fun () =>
            path |> $'System.IO.File.ReadAllTextAsync' |> async.await_task
        | _ => fun () => null ()

()



## file_exists_content

In [ ]:
inl file_exists_content path content : _ bool =
    run_target function
        | Fsharp (Native) => fun () =>
            fun () =>
                fix_condition
                    fun () => path |> file_exists |> not
                    fun () => false |> return
                    fun () =>
                        inl existing_content = path |> read_all_text_async |> async.let'
                        content = existing_content |> return
            |> async.new_async_unit
        | _ => fun () => null ()

()



## write_all_text

In [ ]:
inl write_all_text (path : string) (text : string) : () =
    run_target function
        | Fsharp (Native) => fun () =>
            inl text = join text
            $'System.IO.File.WriteAllText (!path, !text)'
        | Rust (Native) => fun () =>
            open rust_operators
            !\\((path, text), $'"std::fs::write(&*$0, &*$1).unwrap()"')
        | _ => fun () => null ()

()



## read_all_bytes

In [ ]:
inl read_all_bytes (path : string) : am'.vec u8 =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!path |> System.IO.File.ReadAllBytes'
            |> am'.to_vec
        | Rust (Native) => fun () =>
            open rust_operators
            !\\(path, $'"std::fs::read(&*$0).unwrap()"')
        | _ => fun () => null ()

()



## read_all_text

In [ ]:
inl read_all_text (path : string) : string =
    run_target function
        | Fsharp (Native) => fun () =>
            $'!path |> System.IO.File.ReadAllText'
        | Rust (Native) => fun () =>
            path
            |> read_all_bytes
            |> sm'.string_from_utf8
            |> resultm.unwrap'
            |> sm'.from_std_string
        | _ => fun () => null ()

()



## write_all_text_async

In [ ]:
inl write_all_text_async (path : string) (text : string) : _ () =
    run_target function
        | Fsharp (Native) => fun () =>
            $'System.IO.File.WriteAllTextAsync (!path, !text)' |> async.await_task
        | _ => fun () => null ()

()



## write_all_text_exists

In [ ]:
inl write_all_text_exists path contents =
    fun () =>
        inl exists' = contents |> file_exists_content path |> async.let'
        if not exists'
        then contents |> write_all_text_async path |> async.do
    |> async.new_async

()



## delete_directory_async

In [ ]:
inl delete_directory_async path : _ i64 =
    run_target function
        | Fsharp (Native) => fun () =>
            let rec loop (retry : i64) =
                fun () =>
                    try_unit
                        fun () =>
                            path |> directory_delete true
                            retry |> return
                        fun ex =>
                            if retry % 100i64 = 0 then
                                inl ex = ex |> sm'.format_exception
                                trace Debug
                                    fun () => "file_system.delete_directory_async"
                                    fun () => $'$"path: {!path |> !get_file_name} / ex: {!ex} / {!_locals ()}"'
                            async.sleep 10i32 |> async.do
                            loop (retry + 1) |> async.return_await
                |> async.new_async
            loop 0
        | _ => fun () => null ()

()



## create_directory'

In [ ]:
inl create_directory' dir =
    run_target function
        | Fsharp (Native) => fun () =>
            inl directory_info = dir |> create_directory

            inl exists' = directory_info |> directory_info_exists
            if not exists' then
                inl creation_time = directory_info |> directory_info_creation_time
                inl result = ($'{| Exists = !exists'; CreationTime = !creation_time |}' : any) |> sm'.format_debug
                trace Debug
                    fun () => "file_system.create_directory"
                    fun () => $'$"dir: {!dir} / result: {!result} {!_locals ()}"'
            inl disposable : _ () = new_disposable fun () =>
                dir
                |> delete_directory_async
                |> async.ignore
                |> async.run_synchronously
            disposable
        | Rust (Native) => fun () =>
            inl dir = join dir
            match dir |> create_dir_all |> resultm.map_error' sm'.format' |> resultm.unbox with
            | Ok () =>
                trace Verbose
                    fun () => "file_system.create_directory"
                    fun () => $'$"dir: {!dir} / {!_locals ()}"'
            | Error error =>
                trace Critical
                    fun () => "file_system.create_directory"
                    fun () => $'$"dir: {!dir} / error: {!error} / {!_locals ()}"'
            inl disposable : _ () = new_disposable fun () =>
                dir
                |> directory_delete true
            disposable
        | _ => fun () => null ()

()



## create_temp_directory

In [ ]:
inl create_temp_directory () =
    inl dir = create_temp_directory_name ()
    dir, dir |> create_directory'

()



In [ ]:
//// test
//// print_code=false

types ()
inl path, disposable = create_temp_directory ()
disposable |> use |> ignore
path
|> directory_exists
|> _assert_eq true

assert_eq / actual: true / expected: true


In [ ]:
//// test
//// print_code=false
///! rust -d chrono

types ()
inl path, disposable = create_temp_directory ()
path
|> directory_exists
|> _assert_eq true
disposable |> use |> ignore
path
|> directory_exists
|> _assert_eq false

00:00:00 verbose #1 file_system.create_directory / dir: C:\Users\i574n\AppData\Local\Temp\!spiral_builder_9c35be199b6a4ccfecbf1305721883bcbaa9308e71b3562964f9c1c15c4f9620\20240509-0501-1204-5836-00000051b73c
assert_eq / actual: true / expected: true
assert_eq / actual: false / expected: false


In [ ]:
//// test

inl lock_directory path =
    fun () =>
        trace Debug (fun () => "_1") _locals
        "0" |> write_all_text_async (path </> "test.txt") |> async.do
        file_stream
            (path </> "test.txt")
            ModeOpen
            AccessReadWrite
            ShareNone
        |> use
        |> ignore
        trace Debug (fun () => "_2") _locals
        async.sleep 2000 |> async.do
        trace Debug (fun () => "_3") _locals
        () |> return
    |> async.new_async

types ()
inl temp_dir, disposable = create_temp_directory ()
disposable |> use |> ignore
inl path = temp_dir </> "test"

fun () =>
    trace Debug (fun () => "1") _locals
    path |> create_directory |> ignore
    trace Debug (fun () => "2") _locals
    inl child = path |> lock_directory |> async.start_child |> async.let'
    trace Debug (fun () => "3") _locals
    async.sleep 60 |> async.do
    trace Debug (fun () => "4") _locals
    inl retries = path |> delete_directory_async |> async.let'
    trace Debug (fun () => "5") _locals
    child |> async.do
    trace Debug (fun () => "6") _locals
    retries |> return
|> async.new_async_unit
|> async.run_with_timeout 3000
|> fun x => x : _ i64
|> function
    | Some (retries : i64) =>
        retries
        |> _assert_between
            (if runtime.is_windows () then 50 else 0)
            (if runtime.is_windows () then 150 else 0)

        true
    | _ => false
|> _assert_eq true

#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::fs::File")>]
#endif
type std_fs_File = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::fs::FileType")>]
#endif
type std_fs_FileType = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::Display")>]
#endif
type std_path_Display = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::Path")>]
#endif
type std_path_Path = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::PathBuf")>]
#endif
type std_path_PathBuf = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::DirEntry")>]
#endif
type async_walkdir_DirEntry = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::Filtering")>]
#endif
type async_walkdir_Filtering = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::WalkDir")>]
#endif
type async_walkdir_WalkDir = class end
#if FABLE_COMPILER
[<Fabl

## wait_for_file_access

In [ ]:
inl wait_for_file_access access path =
    run_target function
        | Fsharp (Native) => fun () =>
            inl file_access, file_share =
                access
                |> optionm'.default_value (AccessReadWrite, ShareRead)
            let rec loop (retry : i64) : _ i64 =
                fun () =>
                    try_unit
                        fun () =>
                            file_stream
                                path
                                ModeOpen
                                file_access
                                file_share
                            |> use
                            |> ignore
                            retry |> return
                        fun ex =>
                            if retry > 0 && retry % 100i64 = 0 then
                                inl ex = ex |> sm'.format_exception
                                trace Debug
                                    fun () => "file_system.wait_for_file_access"
                                    fun () => $'$"path: {!path |> !get_file_name} / retry: {!retry} / ex: {!ex} / {!_locals ()}"'
                            async.sleep 10i32 |> async.do
                            loop (retry + 1) |> async.return_await
                |> async.new_async
            loop 0
        | _ => fun () => null ()

inl wait_for_file_access_read path =
    path
    |> wait_for_file_access (Some (
        AccessRead,
        ShareRead
    ))

()



In [ ]:
//// test

inl lock_file path =
    fun () =>
        trace Debug (fun () => "_1") _locals
        inl stream : file_stream' =
            file_stream
                path
                ModeOpen
                AccessReadWrite
                ShareNone
            |> use
        trace Debug (fun () => "_2") _locals
        async.sleep 2000 |> async.do
        trace Debug (fun () => "_3") _locals
        ($'!stream.Seek (0L, System.IO.SeekOrigin.Begin)' : i64) |> ignore
        trace Debug (fun () => "_4") _locals
        $'!stream.WriteByte' 49u8
        trace Debug (fun () => "_5") _locals
        stream |> $'_.Flush()'
        trace Debug (fun () => "_6") _locals
    |> async.new_async

types ()
inl temp_dir, disposable = create_temp_directory ()
disposable |> use |> ignore
inl path = temp_dir </> "test.txt"

fun () =>
    trace Debug (fun () => "1") _locals
    "0" |> write_all_text_async path |> async.do
    trace Debug (fun () => "2") _locals
    inl child = path |> lock_file |> async.start_child |> async.let'
    trace Debug (fun () => "3") _locals
    async.sleep 1 |> async.do
    trace Debug (fun () => "4") _locals
    inl retries = path |> wait_for_file_access None |> async.let'
    trace Debug (fun () => "5") _locals
    inl text = path |> read_all_text_async |> async.let'
    trace Debug (fun () => "6") _locals
    child |> async.do
    trace Debug (fun () => "7") _locals
    (retries, text) |> return
|> async.new_async_unit
|> async.run_with_timeout 3000
|> function
    | Some ((retries : i64), text) =>
        retries
        |> _assert_between
            (if runtime.is_windows () then 50 else 100)
            (if runtime.is_windows () then 150 else 200)

        text |> _assert_eq (join "1")

        true
    | _ => false
|> _assert_eq true

#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::fs::File")>]
#endif
type std_fs_File = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::fs::FileType")>]
#endif
type std_fs_FileType = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::Display")>]
#endif
type std_path_Display = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::Path")>]
#endif
type std_path_Path = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::PathBuf")>]
#endif
type std_path_PathBuf = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::DirEntry")>]
#endif
type async_walkdir_DirEntry = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::Filtering")>]
#endif
type async_walkdir_Filtering = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::WalkDir")>]
#endif
type async_walkdir_WalkDir = class end
#if FABLE_COMPILER
[<Fabl

## read_all_text_retry_async

In [ ]:
inl read_all_text_retry_async full_path : async.async (optionm'.option' string) =
    run_target function
        | Fsharp (Native) => fun () =>
            let rec loop (retry : i64) =
                fun () =>
                    try_unit
                        fun () =>
                            if retry > 0
                            then
                                full_path
                                |> wait_for_file_access_read
                                |> async.run_with_timeout_async 1000
                                |> async.ignore
                                |> async.do
                            full_path |> read_all_text_async |> async.map (Some >> optionm'.box) |> async.return_await
                        fun ex =>
                            fix_condition
                                fun () => retry <> 0
                                fun () =>
                                    inl ex = ex |> sm'.format_exception
                                    let _locals () = $'$"retry: {!retry} / ex: {!ex} / {!_locals ()}"'
                                    trace Debug (fun () => $'"read_all_text_retry_async"' : string) _locals
                                    (None : _ string) |> optionm'.box |> return
                                fun () =>
                                    loop (retry + 1) |> async.return_await
                |> async.new_async
            loop 0
        | _ => fun () => null ()

()



## delete_file_async

In [ ]:
inl delete_file_async path : _ i64 =
    run_target function
        | Fsharp (Native) => fun () =>
            let rec loop (retry : i64) =
                fun () =>
                    try_unit
                        fun () =>
                            path |> file_delete
                            return retry
                        fun ex =>
                            if retry % 100 = 0 then
                                trace Warning
                                    fun () => "delete_file_async"
                                    fun () =>
                                        inl ex = ex |> sm'.format_exception
                                        $'$"path: {!path |> !get_file_name} / ex: {!ex} / {!_locals ()}"'
                            async.sleep 10 |> async.do
                            loop (retry + 1) |> async.return_await
                |> async.new_async
            loop 0
        | _ => fun () => null ()

()



In [ ]:
//// test

inl lock_file path =
    fun () =>
        trace Debug (fun () => "_1") _locals
        file_stream
            path
            ModeOpen
            AccessReadWrite
            ShareNone
        |> use
        |> ignore
        trace Debug (fun () => "_2") _locals
        async.sleep 2000 |> async.do
        trace Debug (fun () => "_3") _locals
    |> async.new_async

types ()
inl temp_dir, disposable = create_temp_directory ()
disposable |> use |> ignore
inl path = temp_dir </> "test.txt"

fun () =>
    trace Debug (fun () => "1") _locals
    "0" |> write_all_text_async path |> async.do
    trace Debug (fun () => "2") _locals
    inl child = lock_file path |> async.start_child |> async.let'
    trace Debug (fun () => "3") _locals
    async.sleep 1 |> async.do
    trace Debug (fun () => "4") _locals
    inl retries = delete_file_async path |> async.let'
    trace Debug (fun () => "5") _locals
    child |> async.do
    trace Debug (fun () => "6") _locals
    return retries
|> async.new_async_unit
|> async.run_with_timeout 3000
|> function
    | Some (retries : i64) =>
        retries
        |> _assert_between
            (if runtime.is_windows () then 50 else 0)
            (if runtime.is_windows () then 150 else 0)

        true
    | _ => false
|> _assert_eq true

#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::fs::File")>]
#endif
type std_fs_File = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::fs::FileType")>]
#endif
type std_fs_FileType = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::Display")>]
#endif
type std_path_Display = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::Path")>]
#endif
type std_path_Path = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::PathBuf")>]
#endif
type std_path_PathBuf = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::DirEntry")>]
#endif
type async_walkdir_DirEntry = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::Filtering")>]
#endif
type async_walkdir_Filtering = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::WalkDir")>]
#endif
type async_walkdir_WalkDir = class end
#if FABLE_COMPILER
[<Fabl

## move_file_async

In [ ]:
inl move_file_async new_path old_path : _ i64 =
    run_target function
        | Fsharp (Native) => fun () =>
            let rec loop (retry : i64) =
                fun () =>
                    try_unit
                        fun () =>
                            old_path |> file_move new_path
                            return retry
                        fun ex =>
                            if retry % 100 = 0 then
                                inl _locals () =
                                    inl ex = ex |> sm'.format_exception
                                    $'$"old_path: {!old_path |> !get_file_name} / new_path: {!new_path |> !get_file_name} / ex: {!ex} / {!_locals ()}"'
                                trace Warning (fun () => "move_file_async") _locals
                            async.sleep 10 |> async.do
                            loop (retry + 1) |> async.return_await
                |> async.new_async_unit
            loop 0
        | _ => fun () => null ()

()



In [ ]:
//// test

inl lock_file path =
    fun () =>
        trace Debug (fun () => "_1") _locals
        file_stream
            path
            ModeOpen
            AccessReadWrite
            ShareNone
        |> use
        |> ignore
        trace Debug (fun () => "_2") _locals
        async.sleep 2000 |> async.do
        trace Debug (fun () => "_3") _locals
    |> async.new_async

types ()
inl temp_dir, disposable = create_temp_directory ()
disposable |> use |> ignore
let path = temp_dir </> "test.txt"
let new_path = temp_dir </> "test2.txt"

fun () =>
    trace Debug (fun () => "1") _locals
    "0" |> write_all_text_async path |> async.do
    trace Debug (fun () => "2") _locals
    inl child = lock_file path |> async.start_child |> async.let'
    trace Debug (fun () => "3") _locals
    async.sleep 1 |> async.do
    trace Debug (fun () => "4") _locals
    inl retries1 = path |> move_file_async new_path |> async.let'
    trace Debug (fun () => "5") _locals
    inl retries2 = new_path |> wait_for_file_access None |> async.let'
    trace Debug (fun () => "6") _locals
    inl text = new_path |> read_all_text_async |> async.let'
    trace Debug (fun () => "7") _locals
    child |> async.do
    trace Debug (fun () => "8") _locals
    (retries1, retries2, text) |> return
|> async.new_async_unit
|> async.run_with_timeout 3000
|> function
    | Some (retries1, retries2, text) =>
        retries1
        |> _assert_between
            (if runtime.is_windows () then 50i64 else 0)
            (if runtime.is_windows () then 150 else 0)

        retries2
        |> _assert_between
            (if runtime.is_windows () then 0i64 else 100)
            (if runtime.is_windows () then 0 else 200)

        text |> _assert_eq (join "0")

        true
    | _ => false
|> _assert_eq true

#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::fs::File")>]
#endif
type std_fs_File = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::fs::FileType")>]
#endif
type std_fs_FileType = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::Display")>]
#endif
type std_path_Display = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::Path")>]
#endif
type std_path_Path = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::PathBuf")>]
#endif
type std_path_PathBuf = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::DirEntry")>]
#endif
type async_walkdir_DirEntry = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::Filtering")>]
#endif
type async_walkdir_Filtering = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::WalkDir")>]
#endif
type async_walkdir_WalkDir = class end
#if FABLE_COMPILER
[<Fabl

## find_parent

In [ ]:
inl find_parent name is_file root_dir =
    let rec loop dir =
        if dir </> name |> (if is_file then file_exists else directory_exists)
        then dir
        else
            inl result = dir |> (join directory_get_parent)
            match result |> optionm'.unbox with
            | Some parent => parent |> loop
            | None => failwith $'$"""No parent for {if !is_file then "file" else "dir"} \'{!name}\' at \'{!root_dir}\' (until \'{!dir}\')"""'
    loop root_dir

()



In [ ]:
//// test
//// print_code=false

a ;[ ".paket", false; "paket.dependencies", true ]
|> am.map fun (file, is_file) =>
    get_source_directory ()
    |> find_parent file is_file
    |> directory_info
    |> directory_info_name
|> am'.distinct
|> _assert_eq (a ;[ "polyglot" ] : _ i32 _)

assert_eq / actual: [|"polyglot"|] / expected: [|"polyglot"|]


In [ ]:
//// test
//// print_code=false
///! rust

types ()
a ;[ ".paket", false; "paket.dependencies", true ]
|> am.map fun (file, is_file) =>
    fun () =>
        join
            get_source_directory ()
            |> find_parent file is_file
            |> sm'.to_std_string
            |> new_path_buf
            |> path_buf_file_name
            |> optionm'.try'
            |> sm'.from_os_str_ref
            |> Some
            |> optionm'.box
    |> fun x => x () |> optionm'.unbox
    |> optionm'.default_value ""
|> am'.distinct
|> fun result =>
    result |> am'.length |> _assert_eq 1i32
    index result 0i32 |> _assert_eq "polyglot"

assert_eq / actual: 1 / expected: 1
assert_eq / actual: "polyglot" / expected: "polyglot"


## get_workspace_root

In [ ]:
inl get_workspace_root () =
    get_source_directory ()
    |> find_parent ("polyglot" </> ".paket") false
    |> fun root => root </> "polyglot"

()



## standardize_path

In [ ]:
let standardize_path path =
    path |> get_full_path |> normalize_path

()



## absolute_path

In [ ]:
let absolute_path path =
    inl current_dir = get_current_directory ()
    current_dir </> path |> standardize_path

()



## new_file_uri

In [ ]:
inl new_file_uri (path : string) : string =
    inl path = path |> sm'.trim_start ;[ '/' ]
    $'$"file:///{!path}"'

()



In [ ]:
//// test

@"\\?\C:\test"
|> normalize_path
|> new_file_uri
|> _assert_eq "file:///c:/test"

let rec method1 () : string =
    let v0 : string = "\\\\?\\C:\\test"
    v0
and method2 () : string =
    let v0 : string = ""
    v0
and method3 () : string =
    let v0 : string = "^\\\\\\\\\\?\\\\"
    v0
and method4 () : string =
    let v0 : string = ""
    v0
and method5 (v0 : bool) : bool =
    v0
and method0 () : unit =
    let v0 : bool = true
    let mutable _v0 : string option = None 
    
#if FABLE_COMPILER || WASM || CONTRACT
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v1 : string = method1()
    let v2 : string = method2()
    let v3 : bool = true
    let mutable _v3 : Result<regex_Regex, regex_Error> option = None 
    
#if FABLE_COMPILER || WASM || CONTRACT
    
#if FABLE_COMPILER_RUST && !WASM && !CONTRACT
    let v4 : string = method3()
    let v5 : string = $"regex::Regex::new(&$0)"
    let v6 : Result<regex_Regex, regex_Error> = Fable.Core.RustInterop.emitRustExpr v4 v5
    v6 
    #endif
#if FABLE_COMPILER_RUST && WASM
    let v7 : Result<regex_Rege

In [ ]:
//// test
///! rust -d regex

types ()
@"\\?\C:\test"
|> normalize_path
|> new_file_uri
|> _assert_eq "file:///c:/test"

.rs output:
assert_eq / actual: "file:///c:/test" / expected: "file:///c:/test"




.fsx:
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::fs::File")>]
#endif
type std_fs_File = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::fs::FileType")>]
#endif
type std_fs_FileType = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::Display")>]
#endif
type std_path_Display = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::Path")>]
#endif
type std_path_Path = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::PathBuf")>]
#endif
type std_path_PathBuf = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::DirEntry")>]
#endif
type async_walkdir_DirEntry = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::Filtering")>]
#endif
type async_walkdir_Filtering = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::WalkDir")>]
#endif
type async_walkdir_WalkDir = class end
#if FABLE_COMPILER


## trace_file

In [ ]:
let rec trace_file text =
    run_target function
    | Fsharp (Native) => fun () =>
        try_unit
            fun () =>
                inl assembly_name = env.get_entry_assembly_name ()
                inl guid = date_time.now () |> date_time.new_guid_from_date_time
                inl file_name = $'$"{!assembly_name}_{!guid}.txt"'

                inl workspace_root = get_workspace_root ()
                inl trace_dir = workspace_root </> "target/polyglot/trace"
                trace_dir |> create_directory |> ignore
                inl path = trace_dir </> file_name
                text |> write_all_text_async path |> async.run_synchronously
            fun ex =>
                trace_file $'$"file_system.trace_file / ex: %A{!ex}"'
    | _ => fun () => ()

()



In [ ]:
//// test
//// print_code=false

inl get_count dir : i64 =
    inl files = dir |> file_system.directory_get_files
    a files |> am'.length

types ()
inl trace_dir = file_system.get_workspace_root () </> "target/polyglot/trace"
trace_dir |> file_system.create_directory |> ignore

inl count = get_count trace_dir

trace_file "test"

get_count trace_dir
|> _assert_eq (count + 1)

assert_eq / actual: 47L / expected: 47L


## init_trace_file

In [ ]:
inl init_trace_file enabled =
    inl state_trace_file = get_trace_state () .trace_file
    state_trace_file <- if enabled then trace_file else ignore

()



## main

In [ ]:
inl main () =
    types ()
    init_trace_state ()
    $"let delete_directory_async x = !delete_directory_async x" : ()
    $"let wait_for_file_access x = !wait_for_file_access x" : ()
    $"let wait_for_file_access_read x = !wait_for_file_access_read x" : ()
    $"let read_all_text_async x = !read_all_text_async x" : ()
    $"let file_exists_content x = !file_exists_content x" : ()
    $"let write_all_text_async x = !write_all_text_async x" : ()
    $"let write_all_text_exists x = !write_all_text_exists x" : ()
    $"let delete_file_async x = !delete_file_async x" : ()
    $"let move_file_async x = !move_file_async x" : ()
    $"let read_all_text_retry_async x = !read_all_text_retry_async x" : ()
    $"let create_temp_directory_name () = !create_temp_directory_name ()" : ()
    $"let create_temp_directory () = !create_temp_directory ()" : ()
    $"let get_source_directory () = !get_source_directory ()" : ()
    $"let find_parent x = !find_parent x" : ()
    $"let normalize_path x = !normalize_path x" : ()
    $"let new_file_uri x = !new_file_uri x" : ()
    $"let get_workspace_root () = !get_workspace_root ()" : ()
    $"let init_trace_file x = !init_trace_file x" : ()
    inl combine x = (</>) x
    $"let (</>) x = !combine x" : ()

#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::fs::File")>]
#endif
type std_fs_File = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::fs::FileType")>]
#endif
type std_fs_FileType = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::Display")>]
#endif
type std_path_Display = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::Path")>]
#endif
type std_path_Path = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("std::path::PathBuf")>]
#endif
type std_path_PathBuf = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::DirEntry")>]
#endif
type async_walkdir_DirEntry = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::Filtering")>]
#endif
type async_walkdir_Filtering = class end
#if FABLE_COMPILER
[<Fable.Core.Erase; Fable.Core.Emit("async_walkdir::WalkDir")>]
#endif
type async_walkdir_WalkDir = class end
#if FABLE_COMPILER
[<Fabl